In [2]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit import transpile
from qiskit.visualization import plot_histogram
from qiskit_aer import AerSimulator
import numpy as np
import matplotlib.pyplot as plt

In [5]:
# Ask user for input (0-3) and convert to a 14-bit binary string
target_number = int(input("Enter a number between 0 and 16383 (for 14 bits): "))

# Manually convert decimal to binary using a mathematical method
def decimal_to_binary(n, bits=14):
    binary = []
    while n > 0:
        binary.append(str(n % 2))  # Append the remainder (either 0 or 1)
        n //= 2  # Integer division by 2
    # Add leading zeros if the binary number has fewer than `bits` digits
    while len(binary) < bits:
        binary.append('0')
    return ''.join(binary[::-1])  # Reverse the list to get correct order

# Convert the input number to a 14-bit binary string
target_state = decimal_to_binary(target_number, bits=14)
print(f"Target state: |{target_state}⟩")

def oracle(n, target_state):
    qc = QuantumCircuit(n)
    
    # Apply X gates to flip target state to |11...1⟩ before applying CZ
    for i in range(n):
        if target_state[i] == '0':
            qc.x(i)

    qc.cz(0, 1)  # Phase flip on the marked state (e.g., |11...1⟩)
    
    # Undo the X gates
    for i in range(n):
        if target_state[i] == '0':
            qc.x(i)

    return qc

def diffusion_operator(n, qc):
    # Apply the diffusion operator (inversion about the mean)
    # Apply Hadamard to all qubits
    qc.h(range(n))
    # Apply X to all qubits
    qc.x(range(n))
    # Apply Hadamard to the last qubit
    #qc.h(n-1)
    # Apply CNOT (controlled-X) between the first and second qubit
    qc.cx(0, 1)
    # Apply Hadamard to the last qubit
    #qc.h(n-1)
    # Undo the X and Hadamard gates
    qc.x(range(n))
    qc.h(range(n))

    return qc

n = 14  # Now we are working with 14 qubits
qc = QuantumCircuit(n, n)  # 14 qubits and 14 classical bits for measurement

# Step 1: Apply Hadamard to all qubits (superposition)
qc.h(range(n))

# Step 2: Apply Grover Iterations
iterations = int(np.pi / 4 * np.sqrt(2**n))  # Optimal number of iterations for n=14 qubits
print(f"Number of Grover iterations: {iterations}")

for _ in range(iterations):
    qc.append(oracle(n, target_state), range(n))  # Apply oracle

    # Apply the diffusion operator
    qc = diffusion_operator(n, qc)

# Step 3: Measurement
qc.measure(range(n), range(n))

# Simulate
simulator = AerSimulator()
transpiled_qc = transpile(qc, simulator)
job = simulator.run(transpiled_qc, shots=100000)
result = job.result()
counts = result.get_counts()

# Display results
print("Grover's Algorithm Results (Sorted):")

# Sort the counts from most common to least common
sorted_counts = dict(sorted(counts.items(), key=lambda item: item[1], reverse=True))
print(sorted_counts)

# Manually map binary states to decimal numbers
state_to_decimal = {}
for i in range(2**n):  # For n=14, there are 2^14 = 16384 possible states
    state_to_decimal[format(i, '014b')] = i  # Mapping binary to decimal

# Convert binary states back to decimal using manual mapping
converted_counts = {}
for state, count in sorted_counts.items():
    decimal_state = state_to_decimal[state]  # Manually map the state
    converted_counts[decimal_state] = count

# Display converted results (manual decimal mapping)
print("Converted Results (0-16383) Sorted:")
print(converted_counts)

Target state: |01000101011100⟩
Number of Grover iterations: 100
Grover's Algorithm Results (Sorted):
{'00011100100001': 19, '11100000111011': 18, '01100110001010': 17, '00111011001111': 17, '01001100001000': 17, '11110001011101': 16, '11000011100011': 16, '11111101100100': 16, '11110110101101': 16, '00000101000001': 16, '11001000001010': 16, '01001001010111': 15, '00011101000101': 15, '10010101101110': 15, '00100011011001': 15, '11011101110100': 15, '01110100110010': 15, '11101101101100': 15, '11001101100110': 15, '10010011000100': 15, '11001100011011': 15, '11100101000001': 15, '11000001101000': 15, '01001110010000': 15, '00110111100000': 15, '10110010110000': 15, '10111001010000': 15, '10110100000111': 15, '10011111100011': 15, '10001100010010': 15, '11110110101110': 15, '10101101001000': 15, '00011011101110': 15, '00101100101111': 15, '00101101000100': 15, '00000000101110': 15, '00000110110100': 15, '11001100100111': 14, '11100011010011': 14, '11100010001101': 14, '11010001110110': 